In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, 3) # Input (1 * 28 * 28), Output (26 * 26 * 16)
        self.bn1 = nn.BatchNorm2d(16) # Input (26 * 26 * 16), Output (26 * 26 * 16)

        self.conv2 = nn.Conv2d(16, 16, 3) # Input (26 * 26 * 16), Output (24 * 24 * 16)
        self.bn2 = nn.BatchNorm2d(16) # Input (24 * 24 * 16), Output (24 * 24 * 16)

        self.conv3 = nn.Conv2d(16, 16, 3) # Input (24 * 24 * 16), Output (22 * 22 * 16)
        self.bn3 = nn.BatchNorm2d(16) # Input (22 * 22 * 16), Output (22 * 22 * 16)

        self.pool1 = nn.MaxPool2d(2, 2) # Input (22 * 22 * 16), Output (11 * 11 * 16)

        self.conv4 = nn.Conv2d(16, 16, 3) # Input (11 * 11 * 16), Output (9 * 9 * 16)
        self.bn4 = nn.BatchNorm2d(16) # Input (9 * 9 * 16), Output (9 * 9 * 16)

        self.conv5 = nn.Conv2d(16, 16, 3) # Input (9 * 9 * 16), Output (7 * 7 * 16)
        self.bn5 = nn.BatchNorm2d(16) # Input (7 * 7 * 16), Output (7 * 7 * 16)

        self.conv6 = nn.Conv2d(16, 16, 3) # Input (7 * 7 * 16), Output (5 * 5 * 16)
        self.bn6 = nn.BatchNorm2d(16) # Input (5 * 5 * 16), Output (5 * 5 * 16)

        self.conv7 = nn.Conv2d(16, 32, 3) # Input (5 * 5 * 16), Output (3 * 3 * 32)
        self.bn7 = nn.BatchNorm2d(32) # Input (3 * 3 * 32), Output (3 * 3 * 32)

        self.conv8 = nn.Conv2d(32, 10, 1) # Input (3 * 3 * 32), Output (3 * 3 * 10)

        self.pool2 = nn.AvgPool2d(2) # Input (3 * 3 * 10), Output (1 * 1 * 10)

        self.dense1 = nn.Linear(in_features=1 * 1 * 10, out_features=10)

    def forward(self, x):
        x = self.bn1((F.relu(self.conv1(x))))
        x = F.dropout(self.bn2(F.relu(self.conv2(x))), 0.1)
        x = F.dropout(self.bn3(F.relu(self.conv3(x))), 0.1)
        x = self.pool1(x)
        x = F.dropout(self.bn4(F.relu(self.conv4(x))), 0.1)
        x = F.dropout(self.bn5(F.relu(self.conv5(x))), 0.1)
        x = F.dropout(self.bn6(F.relu(self.conv6(x))), 0.1)
        x = F.dropout(self.bn7(F.relu(self.conv7(x))), 0.1)
        x = self.conv8(x)

        

        x = self.pool2(x)        
        x = x.view(-1, 1 * 1 * 10)        
        x = self.dense1(x)
        
        
        return F.log_softmax(x)
        
        

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
            Conv2d-3           [-1, 16, 24, 24]           2,320
       BatchNorm2d-4           [-1, 16, 24, 24]              32
            Conv2d-5           [-1, 16, 22, 22]           2,320
       BatchNorm2d-6           [-1, 16, 22, 22]              32
         MaxPool2d-7           [-1, 16, 11, 11]               0
            Conv2d-8             [-1, 16, 9, 9]           2,320
       BatchNorm2d-9             [-1, 16, 9, 9]              32
           Conv2d-10             [-1, 16, 7, 7]           2,320
      BatchNorm2d-11             [-1, 16, 7, 7]              32
           Conv2d-12             [-1, 16, 5, 5]           2,320
    

<ipython-input-2-5c1abe464ffb>:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm

target_accuracy = 99.4

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    print('\nEpoch: {}, Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_accuracy, test_loss

    '''
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    '''

In [6]:

model = Net().to(device)
#optimizer = optim.SGD(model.parameters(), lr=0.013, momentum=0.9)
optimizer= optim.Adam(model.parameters(), lr= 0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2) 

best_test_accuracy = 0.0
targetAchieved = False


for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test_accuracy_local, test_loss_local = test(model, device, test_loader, epoch)

    if test_accuracy_local > best_test_accuracy:
      best_test_accuracy = test_accuracy_local

    if test_accuracy_local >= target_accuracy:
      print('\nTarget Accuracy obtained in {} epochs. Current Test Accuracy is {:.2f}%. Stopping the Program.'.format(epoch, test_accuracy_local))
      best_test_accuracy = test_accuracy_local
      targetAchieved = True
      break
    
    scheduler.step(test_loss_local)

if not targetAchieved:
  print('Best Accuracy Observed as {:.2f}% while within 20 epochs.'.format(best_test_accuracy))

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-5c1abe464ffb>:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.04569244012236595 batch_id=468: 100%|██████████| 469/469 [01:26<00:00,  5.40it/s]



Epoch: 1, Test set: Average loss: 0.0555, Accuracy: 9828/10000 (98.28%)



loss=0.011702985502779484 batch_id=468: 100%|██████████| 469/469 [01:25<00:00,  5.46it/s]



Epoch: 2, Test set: Average loss: 0.0567, Accuracy: 9827/10000 (98.27%)



loss=0.0264867153018713 batch_id=468: 100%|██████████| 469/469 [01:25<00:00,  5.47it/s]


...


Epoch: 13, Test set: Average loss: 0.0411, Accuracy: 9886/10000 (98.86%)



loss=0.10376865416765213 batch_id=468: 100%|██████████| 469/469 [01:25<00:00,  5.48it/s]



Epoch: 14, Test set: Average loss: 0.0235, Accuracy: 9938/10000 (99.38%)



loss=0.019813166931271553 batch_id=468: 100%|██████████| 469/469 [01:26<00:00,  5.42it/s]



Epoch: 15, Test set: Average loss: 0.0226, Accuracy: 9944/10000 (99.44%)


Target Accuracy obtained in 15 epochs. Current Test Accuracy is 99.44%. Stopping the Program.
